In [1]:
#!/usr/bin/env python

from __future__ import division
import os
import math
import re
import sys



In [ ]:

def re_replace_constant(string, typename, varname, value):
    constant_re = re.compile(
        r"({} +constant +{} +=) +(.*);".format(typename, varname)
    )
    match = constant_re.search(string)
    if not match:
        print(
            "ERROR: Could not match RE for '{}' during template generation.".
            format(varname)
        )
        sys.exit(1)

    if match.groups()[1] == str(value):
        # The value already exists in the source
        return string

    new_string = constant_re.sub(r"\1 {};".format(str(value)), string)
    return new_string

In [2]:
def re_replace_constant_and_type(string, typename, varname, value):
    constant_re = re.compile(
        r"( *)(.*) +constant +({}) += +(.*);".format(varname)
    )
    match = constant_re.search(string)
    if not match:
        print(
            "ERROR: Could not match RE for '{}' during template generation.".
            format(varname)
        )
        sys.exit(1)

    if match.groups()[1] == typename and match.groups()[3] == str(value):
        # The variable already exists in the source as we want it
        return string

    new_string = constant_re.sub(
        r"\1{} constant \3 = {};".format(typename, str(value)),
        string
    )
    return new_string

In [3]:
def re_replace_vardecl(string, typename, varname):
    var_re = re.compile(
        r"( *)(uint.*) +({});".format(varname)
    )
    match = var_re.search(string)
    if not match:
        print(
            "ERROR: Could not match RE for '{}' during template generation.".
            format(varname)
        )
        sys.exit(1)

    if match.groups()[1] == typename:
        # The variable already exists in the source as we want it
        return string

    new_string = var_re.sub(
        r"\1{} {};".format(typename, varname),
        string
    )
    return new_string

In [4]:
def re_replace_function_params(string, func_name, param_type):
    func_re = re.compile(
        r"( *)function {}\((.*?) *_angle\)".format(func_name)
    )
    match = func_re.search(string)
    if not match:
        print(
            "ERROR: Could not match function '{}' during template generation.".
            format(func_name)
        )
        sys.exit(1)

    if match.groups()[1] == param_type:
        # The type already exists in the source as we want it
        return string

    new_string = func_re.sub(
        r"\1function {}({} _angle)".format(func_name, param_type),
        string
    )
    return new_string

In [5]:
def re_replace_function_return(string, func_name, param_type):
    func_re = re.compile(
        r"( *)function {}\((.*)\)(.*)returns *\((.*?)\)".format(func_name)
    )
    match = func_re.search(string)
    if not match:
        print(
            "ERROR: Could not match function '{}' during template generation.".
            format(func_name)
        )
        sys.exit(1)

    if match.groups()[3] == param_type:
        # The type already exists in the source as we want it
        return string

    new_string = func_re.sub(
        r"\1function {}(\2)\3returns ({})".format(func_name, param_type),
        string
    )
    return new_string


In [6]:
def re_replace_comments(string, number_of_bits):
    angles_in_circle = 1 << (number_of_bits - 2)
    amplitude = (1 << (number_of_bits - 1)) - 1
    comment_re = re.compile(
        r'( *)\* @param _angle A (\d+)-bit angle. This divides'
        ' the circle into (\d+)'
    )
    match = comment_re.search(string)
    if not match:
        print(
            "ERROR: Could not match angle comment during template generation."
        )
        sys.exit(1)
    if (
            int(match.groups()[1]) != number_of_bits - 2
            or int(match.groups()[2]) == angles_in_circle
    ):
        new_string = comment_re.sub(
            r'\1* @param _angle A {}-bit angle. This divides'
            'the circle into {}'.format(
                str(number_of_bits - 2),
                angles_in_circle
            ),
            string
        )

    string = new_string
    comment_re = re.compile(
        r'( *)\* @return The sine result as a number in the range '
        '-(\d+) to (\d+)'
    )
    if not match:
        print(
            "ERROR: Could not match return comment during template generation."
        )
        sys.exit(1)
    if (
            int(match.groups()[1]) == amplitude
            and int(match.groups()[2]) == amplitude
    ):
        # The comment already exists in the source as we want it
        return string

    new_string = comment_re.sub(
        r'\1* @return The sine result as a number '
        'in the range -{} to {}'.format(
            amplitude,
            amplitude,
        ),
        string
    )

    return new_string

In [10]:
arr = []

def gen_sin_table(number_of_bits, table_size):
    table = '"'
    number_of_bytes = int(number_of_bits / 8)
    amplitude = (1 << (number_of_bits - 1)) - 1
#     print(amplitude)
    for i in range(0, table_size):
        radians = (i * (math.pi / 2)) / (table_size - 1)
        sin_value = amplitude * math.sin(radians)
        table_value = round(sin_value)
        hex_value = "{0:0{1}x}".format(int(table_value), 2 * number_of_bytes)
        print(table_value, hex_value)
        arr.append(table_value)
        table += '\\x' + '\\x'.join(
            hex_value[i: i + 2] for i in range(0, len(hex_value), 2)
        )
    return table + '"'

In [267]:
def bytes_to_int(bytes):
    result = 0
    for b in bytes:
        result = result * 256 + int(b)
    return result

In [249]:
number_of_bits = 16
table_size = (2 ** int(number_of_bits / 4)) + 1

In [250]:
gen_sin_table(number_of_bits, table_size)


0 0000
3212 0c8c
6393 18f9
9512 2528
12539 30fb
15446 3c56
18204 471c
20787 5133
23170 5a82
25329 62f1
27245 6a6d
28898 70e2
30273 7641
31356 7a7c
32137 7d89
32609 7f61
32767 7fff


'"\\x00\\x00\\x0c\\x8c\\x18\\xf9\\x25\\x28\\x30\\xfb\\x3c\\x56\\x47\\x1c\\x51\\x33\\x5a\\x82\\x62\\xf1\\x6a\\x6d\\x70\\xe2\\x76\\x41\\x7a\\x7c\\x7d\\x89\\x7f\\x61\\x7f\\xff"'

In [251]:
"\x00\x00\x0c\x8c\x18\xf9\x25\x28\x30\xfb\x3c\x56\x47\x1c\x51\x33\x5a\x82\x62\xf1\x6a\x6d\x70\xe2\x76\x41\x7a\x7c\x7d\x89\x7f\x61\x7f\xff"




'\x00\x00\x0c\x8c\x18ù%(0û<VG\x1cQ3Z\x82bñjmpâvAz|}\x89\x7fa\x7fÿ'

In [252]:
bytes.fromhex('0c8c18f9')

b'\x0c\x8c\x18\xf9'

In [253]:
import numpy as np
import math

In [248]:
# math.factorial(10)

In [254]:
def lookup_fact(N):
    arr = []
    s = ''
    j = 1
    for i in range(0, N):
        j += i%2
        fact_value = math.factorial(i)
        table_value = round(fact_value)
        n = (math.floor(math.log(fact_value, 256)) + 1)
        arr.append(((table_value).to_bytes(n, 'big')))
#         print(arr[-1], table_value, n)
    print(arr)
    return b''.join(arr)

In [255]:
n = (math.floor(math.log(479001600, 256)) + 1)

(479001600).to_bytes(n, 'big')

b'\x1c\x8c\xfc\x00'

In [72]:
hex(32767)

'0x7fff'

In [257]:
# bytes.fromhex('0x130777758000')

In [185]:
((1).to_bytes(2, 'big')).strip()

b'\x00\x01'

In [239]:
((lookup_fact(6))[:])

[b'\x01', b'\x01', b'\x02', b'\x06', b'\x18', b'x']


b'\x01\x01\x02\x06\x18x'

In [235]:
len("00_00_00_00_00_c9_0f_88_01_92_1d_20_02_5b_26_d7_03_24_2a_bf_03_ed_26_e6_04_b6_19_5d_05_7f_00_35_06_47_d9_7c_07_10_a3_45_07_d9_5b_9e_08_a2_00_9a_09_6a_90_49_0a_33_08_bc_0a_fb_68_05_0b_c3_ac_35_0c_8b_d3_5e_0d_53_db_92_0e_1b_c2_e4_0e_e3_87_66_0f_ab_27_2b_10_72_a0_48_11_39_f0_cf_12_01_16_d5_12_c8_10_6e_13_8e_db_b1_14_55_76_b1_15_1b_df_85_15_e2_14_44_16_a8_13_05_17_6d_d9_de_18_33_66_e8_18_f8_b8_3c_19_bd_cb_f3_1a_82_a0_25_1b_47_32_ef_1c_0b_82_6a_1c_cf_8c_b3_1d_93_4f_e5_1e_56_ca_1e_1f_19_f9_7b_1f_dc_dc_1b_20_9f_70_1c_21_61_b3_9f_22_23_a4_c5_22_e5_41_af_23_a6_88_7e_24_67_77_57_25_28_0c_5d_25_e8_45_b6_26_a8_21_85_27_67_9d_f4_28_26_b9_28_28_e5_71_4a_29_a3_c4_85_2a_61_b1_01_2b_1f_34_eb_2b_dc_4e_6f_2c_98_fb_ba_2d_55_3a_fb_2e_11_0a_62_2e_cc_68_1e_2f_87_52_62_30_41_c7_60_30_fb_c5_4d_31_b5_4a_5d_32_6e_54_c7_33_26_e2_c2_33_de_f2_87_34_96_82_4f_35_4d_90_56_36_04_1a_d9_36_ba_20_13_37_6f_9e_46_38_24_93_b0_38_d8_fe_93_39_8c_dd_32_3a_40_2d_d1_3a_f2_ee_b7_3b_a5_1e_29_3c_56_ba_70_3d_07_c1_d5_3d_b8_32_a5_3e_68_0b_2c_3f_17_49_b7_3f_c5_ec_97_40_73_f2_1d_41_21_58_9a_41_ce_1e_64_42_7a_41_d0_43_25_c1_35_43_d0_9a_ec_44_7a_cd_50_45_24_56_bc_45_cd_35_8f_46_75_68_27_47_1c_ec_e6_47_c3_c2_2e_48_69_e6_64_49_0f_57_ee_49_b4_15_33_4a_58_1c_9d_4a_fb_6c_97_4b_9e_03_8f_4c_3f_df_f3_4c_e1_00_34_4d_81_62_c3_4e_21_06_17_4e_bf_e8_a4_4f_5e_08_e2_4f_fb_65_4c_50_97_fc_5e_51_33_cc_94_51_ce_d4_6e_52_69_12_6e_53_02_85_17_53_9b_2a_ef_54_33_02_7d_54_ca_0a_4a_55_60_40_e2_55_f5_a4_d2_56_8a_34_a9_57_1d_ee_f9_57_b0_d2_55_58_42_dd_54_58_d4_0e_8c_59_64_64_97_59_f3_de_12_5a_82_79_99_5b_10_35_ce_5b_9d_11_53_5c_29_0a_cc_5c_b4_20_df_5d_3e_52_36_5d_c7_9d_7b_5e_50_01_5d_5e_d7_7c_89_5f_5e_0d_b2_5f_e3_b3_8d_60_68_6c_ce_60_ec_38_2f_61_6f_14_6b_61_f1_00_3e_62_71_fa_68_62_f2_01_ac_63_71_14_cc_63_ef_32_8f_64_6c_59_bf_64_e8_89_25_65_63_bf_91_65_dd_fb_d2_66_57_3c_bb_66_cf_81_1f_67_46_c7_d7_67_bd_0f_bc_68_32_57_aa_68_a6_9e_80_69_19_e3_1f_69_8c_24_6b_69_fd_61_4a_6a_6d_98_a3_6a_dc_c9_64_6b_4a_f2_78_6b_b8_12_d0_6c_24_29_5f_6c_8f_35_1b_6c_f9_34_fb_6d_62_27_f9_6d_ca_0d_14_6e_30_e3_49_6e_96_a9_9c_6e_fb_5f_11_6f_5f_02_b1_6f_c1_93_84_70_23_10_99_70_83_78_fe_70_e2_cb_c5_71_41_08_04_71_9e_2c_d1_71_fa_39_48_72_55_2c_84_72_af_05_a6_73_07_c3_cf_73_5f_66_25_73_b5_eb_d0_74_0b_53_fa_74_5f_9d_d0_74_b2_c8_83_75_04_d3_44_75_55_bd_4b_75_a5_85_ce_75_f4_2c_0a_76_41_af_3c_76_8e_0e_a5_76_d9_49_88_77_23_5f_2c_77_6c_4e_da_77_b4_17_df_77_fa_b9_88_78_40_33_28_78_84_84_13_78_c7_ab_a1_79_09_a9_2c_79_4a_7c_11_79_8a_23_b0_79_c8_9f_6d_7a_05_ee_ac_7a_42_10_d8_7a_7d_05_5a_7a_b6_cb_a3_7a_ef_63_23_7b_26_cb_4e_7b_5d_03_9d_7b_92_0b_88_7b_c5_e2_8f_7b_f8_88_2f_7c_29_fb_ed_7c_5a_3d_4f_7c_89_4b_dd_7c_b7_27_23_7c_e3_ce_b1_7d_0f_42_17_7d_39_80_eb_7d_62_8a_c5_7d_8a_5f_3f_7d_b0_fd_f7_7d_d6_66_8e_7d_fa_98_a7_7e_1d_93_e9_7e_3f_57_fe_7e_5f_e4_92_7e_7f_39_56_7e_9d_55_fb_7e_ba_3a_38_7e_d5_e5_c5_7e_f0_58_5f_7f_09_91_c3_7f_21_91_b3_7f_38_57_f5_7f_4d_e4_50_7f_62_36_8e_7f_75_4e_7f_7f_87_2b_f2_7f_97_ce_bc_7f_a7_36_b3_7f_b5_63_b2_7f_c2_55_95_7f_ce_0c_3d_7f_d8_87_8d_7f_e1_c7_6a_7f_e9_cb_bf_7f_f0_94_77_7f_f6_21_81_7f_fa_72_d0_7f_fd_88_59_7f_ff_62_15_7f_ff_ff_ff".split('_'))

1028

In [247]:
1000 >> 8

3

In [259]:
index = 15 # want to know "15!"


(lookup_fact(16))[:]

[b'\x01', b'\x01', b'\x02', b'\x06', b'\x18', b'x', b'\x02\xd0', b'\x13\xb0', b'\x9d\x80', b'\x05\x89\x80', b'7_\x00', b'\x02a\x15\x00', b'\x1c\x8c\xfc\x00', b'\x01s(\xcc\x00', b'\x14L;(\x00', b'\x010wwX\x00']


b'\x01\x01\x02\x06\x18x\x02\xd0\x13\xb0\x9d\x80\x05\x89\x807_\x00\x02a\x15\x00\x1c\x8c\xfc\x00\x01s(\xcc\x00\x14L;(\x00\x010wwX\x00'

In [260]:
bytes_to_int(bytes.fromhex('00000c8c18f9')[4:6])

6393

In [261]:
(1 << (number_of_bits - 1)) 

32768

In [262]:
int('18f9', 16)

6393

In [264]:
# bytes_to_int(sin_table[:3]) % 2**15

In [281]:
# magic:

x = bytes(3) + b'\x11' + bytes(1)

print(x)
print('-------')
print(bytes_to_int(x))

b'\x00\x00\x00\x11\x00'
-------
4352


In [284]:
# int.from_bytes(sin_table[124:126], 'big')

In [287]:
bin(2958)

'0b101110001110'

In [200]:
hex(2958)

'0xb8e'

In [64]:
import numpy as np

In [66]:
np.log2(1073741824)

30.0

In [8]:
def generate_trigonometry(number_of_bits, for_tests):
    print("Generating the sin() lookup table ...")
    table_size = (2 ** int(number_of_bits / 4)) + 1
    if number_of_bits % 8 != 0:
        print("ERROR: Bits should be a multiple of 8")
        sys.exit(1)

    path = os.path.join(
        os.path.dirname(os.path.dirname(os.path.abspath(__file__))),
        'contracts',
        'trigonometry.sol'
    )
    with open(path) as f:
        lines = f.read()

    uint_type_name = 'uint' + str(number_of_bits)
    lines = re_replace_constant(
        lines,
        'uint8',
        'entry_bytes',
        int(number_of_bits / 8)
    )
    lines = re_replace_constant(
        lines,
        'uint',
        'INDEX_WIDTH',
        int(number_of_bits / 4)
    )
    lines = re_replace_constant(
        lines,
        'uint',
        'INTERP_WIDTH',
        int(number_of_bits / 2)
    )
    lines = re_replace_constant(
        lines,
        'uint',
        'INDEX_OFFSET',
        '{} - INDEX_WIDTH'.format(number_of_bits - 4)
    )
    lines = re_replace_vardecl(lines, uint_type_name, 'trigint_value')
    lines = re_replace_constant_and_type(
        lines,
        uint_type_name,
        'ANGLES_IN_CYCLE',
        1 << (number_of_bits - 2)
    )
    lines = re_replace_constant(
        lines,
        'uint',
        'SINE_TABLE_SIZE',
        table_size - 1
    )
    lines = re_replace_constant_and_type(
        lines,
        uint_type_name,
        'QUADRANT_HIGH_MASK',
        int(1 << (number_of_bits - 3))
    )
    lines = re_replace_constant_and_type(
        lines,
        uint_type_name,
        'QUADRANT_LOW_MASK',
        int(1 << (number_of_bits - 4))
    )
    lines = re_replace_constant(
        lines,
        'bytes',
        'sin_table',
        gen_sin_table(number_of_bits, table_size)
    )
    lines = re_replace_function_params(
        lines,
        'sin',
        uint_type_name
    )
    lines = re_replace_function_params(
        lines,
        'cos',
        uint_type_name
    )
    lines = re_replace_function_return(
        lines,
        'sin_table_lookup',
        uint_type_name
    )
    lines = re_replace_comments(lines, number_of_bits)

    if for_tests:
        path = os.path.join(
            os.path.dirname(os.path.dirname(os.path.abspath(__file__))),
            'contracts',
            'trigonometry_generated.sol'
        )
        lines = lines.replace(
            'library Trigonometry',
            'library TrigonometryGenerated'
        )

    with open(path, 'w') as f:
        f.write(lines)

In [10]:


if __name__ == '__main__':
    number_of_bits = 16
    generate_trigonometry(number_of_bits, for_tests=False)

Generating the sin() lookup table ...


NameError: name '__file__' is not defined